$\large\text{Generate folders, files and run batches}$

In [14]:
import subprocess
import os
import time

In [15]:
vtkpath = "/home/gabriel/Desktop/ProjectFiles/ColloidalMembranes/VTKMega/Circular/ByRadius"
initialmeshpath = "/home/gabriel/Desktop/ProjectFiles/ColloidalMembranes/VTKMega/Circular/ByRadius"
snapshotpath = "/home/gabriel/Desktop/ProjectFiles/ColloidalMembranes/CodeMega/DataAnalysis/Circular/ByRadius/FinalSnapshot"
childpath = "/home/gabriel/Desktop/ProjectFiles/ColloidalMembranes/CodeMega/MorphoSimulations/Circular/ByRadius"
parentpath = "/home/gabriel/Desktop/ProjectFiles/ColloidalMembranes/CodeMega/MorphoSimulations"
initialmeshfilehead = "CircularRegionPoreMeshGen.morpho"
parentfilehead = "CircularRegionPoreMain.morpho" 

$\text{Define parameters for the batch:}$

In [16]:
# qchirality = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9]
qchirality = [0.6, 0.8, 1.0, 1.2, 1.4, 1.6]
poreradius = [0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75]
porestartingradius = [0.35, 0.45, 0.55, 0.65, 0.75, 0.75, 0.75, 0.75]
#[0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2]
# glinetension = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

$\text{Create new folders for morpho files and VTK exports}$

In [17]:
if not os.path.exists(vtkpath):
    os.makedirs(vtkpath)
    
if not os.path.exists(snapshotpath):
    os.makedirs(snapshotpath)
    
if not os.path.exists(childpath):
    os.makedirs(childpath)

for q in qchirality:
    for r in poreradius:
        newpath = vtkpath + f'/q={q} r={r}' 
        if not os.path.exists(newpath):
            os.makedirs(newpath)

In [18]:
# for q in qchirality:
#     for g in glinetension:
#         newpath = childpath + f'/q={q} g={g}' 
#         if os.path.exists(newpath):
#             ldocs = os.listdir(newpath)
#             for st in ldocs:
#                 os.remove(newpath+"/"+st)

$\text{Create and write to the new morpho files}$

In [19]:
def overwritemeshgen(parentfile, childfile, r, rs):
            with open(parentfile, 'r') as file:
                # read a list of lines into data
                data = file.readlines()

            data[15] = f"var Rp = {r}/lamt \n"
            data[16] = f"var Rpst = {rs}/lamt\n"
            data[22] = f'var pathexport = "{initialmeshpath}"\n'
            # and write everything back
            with open(childfile, 'w') as newfile:
                newfile.writelines( data )

for i in range(0, len(poreradius)):
    childfile = initialmeshpath + f'/CircularRegionPoreMeshGenRp{poreradius[i]}.morpho'
    parentfile = parentpath+'/'+initialmeshfilehead
    overwritemeshgen(parentfile, childfile, poreradius[i], porestartingradius[i])

In [20]:
def overwritemain(parentfile, childfile, q, r):
            with open(parentfile, 'r') as file:
                # read a list of lines into data
                data = file.readlines()

            data[17] = f"var qstar = {q} \n"
            data[24] = f"var Rp = {r} \n"
            data[29] = f'var pathexport = "{vtkpath}/q={q} r={r}"\n'
            data[30] = f'var initialmeshpath = "{initialmeshpath}"\n'
            # and write everything back
            with open(childfile, 'w') as newfile:
                newfile.writelines( data )

for q in qchirality:
    for r in poreradius:
        childfile = childpath + f'/CircularRegionPoreMainRp{r}q{q}.morpho'
        parentfile = parentpath+'/'+parentfilehead
        overwritemain(parentfile, childfile, q, r)

$\text{Call the new files in ssh terminal}$

In [25]:
ti = time.time()
for r in poreradius:
    t1 = time.time()
    file = initialmeshpath + f'/CircularRegionPoreMeshGenRp{r}.morpho'
    call = subprocess.call(['morpho6','-w8', file])
    t2 = time.time()
    print(f"call: {call} r = {r}, time of mesh generation = {(t2 - t1)/60} min \n")
tf = time.time()
print(f"time of batch: {(tf - ti)/60} min")

Creating initial mesh on regular grid
Retriangulating.
Mesh generator iteration 0.
Iteration 0. Energy: 286.769 delta E: 6.16658 stepsize: 0.0305207
Iteration 0. Energy: 283.67 delta E: 3.09834 stepsize: 0.0305207
Retriangulating.
Mesh generator iteration 1.
Iteration 0. Energy: 279.987 delta E: 3.60303 stepsize: 0.0555893
Iteration 0. Energy: 277.593 delta E: 2.39386 stepsize: 0.0555893
Retriangulating.
Mesh generator iteration 2.
Iteration 0. Energy: 265.651 delta E: 1.95332 stepsize: 0.0731113
Iteration 0. Energy: 264.61 delta E: 1.04145 stepsize: 0.0257987
Retriangulating.
Mesh generator iteration 3.
Iteration 0. Energy: 254.669 delta E: 0.758069 stepsize: 0.0467652
Iteration 0. Energy: 254.059 delta E: 0.609894 stepsize: 0.0249485
Retriangulating.
Mesh generator iteration 4.
Iteration 0. Energy: 249.022 delta E: 0.567625 stepsize: 0.0452838
Iteration 0. Energy: 248.503 delta E: 0.519248 stepsize: 0.0243713
Retriangulating.
Mesh generator iteration 5.
Iteration 0. Energy: 242.755 d

In [21]:
print(f"Starting batch of "+parentfilehead+f" \n Batch size: {len(qchirality)} x {len(poreradius)} \n qs = {qchirality} \n rs = {poreradius} \n")

Starting batch of CircularRegionPoreMain.morpho 
 Batch size: 6 x 8 
 qs = [0.6, 0.8, 1.0, 1.2, 1.4, 1.6] 
 rs = [0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75] 



In [22]:
ti = time.time()
for q in qchirality:
    for r in poreradius:
        t1 = time.time()
        file = childpath + f'/CircularRegionPoreMainRp{r}q{q}.morpho'
        call = subprocess.call(['morpho6','-w8', file])
        t2 = time.time()
        print(f"call: {call} q = {q}, r = {r}, time of simulation = {(t2 - t1)/60} min \n")
tf = time.time()
print(f"time of batch: {(tf - ti)/60} min")

Error 'FlOpnFld': Couldn't open file '/home/gabriel/Desktop/ProjectFiles/ColloidalMembranes/VTKMega/Circular/ByRadius/InitialMeshCircular.vtk'.
  from init at line 381 in module '/usr/local/share/morpho/modules/vtk.morpho'
  from global at line 251
call: 0 q = 0.6, r = 0.05, time of simulation = 0.0006406426429748536 min 

Error 'FlOpnFld': Couldn't open file '/home/gabriel/Desktop/ProjectFiles/ColloidalMembranes/VTKMega/Circular/ByRadius/InitialMeshCircular.vtk'.
  from init at line 381 in module '/usr/local/share/morpho/modules/vtk.morpho'
  from global at line 251
call: 0 q = 0.6, r = 0.15, time of simulation = 0.00041248798370361326 min 

Error 'FlOpnFld': Couldn't open file '/home/gabriel/Desktop/ProjectFiles/ColloidalMembranes/VTKMega/Circular/ByRadius/InitialMeshCircular.vtk'.
  from init at line 381 in module '/usr/local/share/morpho/modules/vtk.morpho'
  from global at line 251
call: 0 q = 0.6, r = 0.25, time of simulation = 0.00036872227986653645 min 

Error 'FlOpnFld': Could